In [2]:
import pandas as pd

In [70]:
dfcharge = pd.read_csv("charging_sessions.csv")
dfweather = pd.read_csv("weather_burbank_airport.csv")

In [92]:
dfcharge.head()

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,250.0,25.0,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,280.0,70.0,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,400.0,8.0,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,400.0,8.0,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,400.0,8.0,20.0,65.0,"Thu, 02 Jan 2020 14:00:19 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"


In [71]:
import json

#Json format needs lowercase boolean values and " instead of '
dfcharge['userInputs'] = dfcharge['userInputs'].apply(lambda x: x.replace("True", "true").replace("False", "false").replace("'", "\"") if isinstance(x, str) else x)

normalized_data = []
#iterate over dfcharge rows and convert the json entries to a list and then add them to their respective column
for index, row in dfcharge.iterrows():
    if pd.notna(row['userInputs']):
        json_data_list = json.loads(row['userInputs'])
        for json_data in json_data_list:
            normalized_data.append({**row, **json_data})
    else:
        normalized_data.append(row)

dfcharge = pd.DataFrame(normalized_data).drop("userInputs",axis=1)

In [101]:
dfcharge["connectionTime"] = pd.to_datetime(dfcharge["connectionTime"]).dt.tz_localize(None)
dfcharge["disconnectTime"] = pd.to_datetime(dfcharge["disconnectTime"]).dt.tz_localize(None)
dfcharge["doneChargingTime"] = pd.to_datetime(dfcharge["doneChargingTime"]).dt.tz_localize(None)
dfcharge["modifiedAt"] = pd.to_datetime(dfcharge["modifiedAt"]).dt.tz_localize(None)
dfcharge["requestedDeparture"] = pd.to_datetime(dfcharge["requestedDeparture"]).dt.tz_localize(None)
dfweather["timestamp"] = pd.to_datetime(dfweather["timestamp"]).dt.tz_localize(None)

In [104]:
dfcharge = dfcharge.sort_values(by="connectionTime")
merged_df = pd.merge_asof(dfcharge, dfweather, left_on='connectionTime', right_on='timestamp')

In [105]:
merged_df

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,...,requestedDeparture,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
0,0,5bc90cb9f9af8b0d7fe77cd2,2018-04-25 11:08:04,2018-04-25 13:20:10,2018-04-25 13:21:10,7.932,2_39_78_362_2018-04-25 11:08:04.400812,2,CA-496,2-39-78-362,...,NaT,Burbank,2018-04-25 10:53:00,12.0,26.0,Cloudy,989.11,11.0,0.0,12.0
1,1,5bc90cb9f9af8b0d7fe77cd3,2018-04-25 13:45:10,2018-04-26 00:56:16,2018-04-25 16:44:15,10.013,2_39_95_27_2018-04-25 13:45:09.617470,2,CA-319,2-39-95-27,...,NaT,Burbank,2018-04-25 13:24:00,12.0,20.0,Fog,989.44,9.0,0.0,12.0
2,2,5bc90cb9f9af8b0d7fe77cd4,2018-04-25 13:45:50,2018-04-25 23:04:45,2018-04-25 14:51:44,5.257,2_39_79_380_2018-04-25 13:45:49.962001,2,CA-489,2-39-79-380,...,NaT,Burbank,2018-04-25 13:24:00,12.0,20.0,Fog,989.44,9.0,0.0,12.0
3,3,5bc90cb9f9af8b0d7fe77cd5,2018-04-25 14:37:06,2018-04-25 23:55:34,2018-04-25 16:05:22,5.177,2_39_79_379_2018-04-25 14:37:06.460772,2,CA-327,2-39-79-379,...,NaT,Burbank,2018-04-25 13:53:00,12.0,20.0,Fog,989.44,7.0,0.0,12.0
4,4,5bc90cb9f9af8b0d7fe77cd6,2018-04-25 14:40:34,2018-04-25 23:03:12,2018-04-25 17:40:30,10.119,2_39_79_381_2018-04-25 14:40:33.638896,2,CA-490,2-39-79-381,...,NaT,Burbank,2018-04-25 13:53:00,12.0,20.0,Fog,989.44,7.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80634,3031,61550519f9af8b76960e169c,2021-09-13 22:33:07,2021-09-13 23:06:55,NaT,17.720,2_39_81_4550_2021-09-13 22:33:04.543952,2,11900388,2-39-81-4550,...,2021-09-14 03:21:07,Burbank,2021-01-01 07:53:00,10.0,33.0,Fair,987.80,6.0,0.0,10.0
80635,3032,61550519f9af8b76960e169d,2021-09-13 23:11:12,2021-09-14 01:43:11,2021-09-14 00:13:35,2.018,2_39_91_437_2021-09-13 23:10:59.528292,2,CA-317,2-39-91-437,...,2021-09-14 01:11:12,Burbank,2021-01-01 07:53:00,10.0,33.0,Fair,987.80,6.0,0.0,10.0
80636,5874,6155053bf9af8b76960e16d0,2021-09-14 01:08:16,2021-09-14 01:31:24,2021-09-14 01:31:19,1.253,1_1_179_783_2021-09-14 01:08:03.220292,1,AG-3F29,1-1-179-783,...,2021-09-14 05:27:16,Burbank,2021-01-01 07:53:00,10.0,33.0,Fair,987.80,6.0,0.0,10.0
80637,3033,61550519f9af8b76960e169e,2021-09-14 01:52:37,2021-09-14 03:05:10,NaT,45.064,2_39_81_4550_2021-09-14 01:52:31.129163,2,11900388,2-39-81-4550,...,2021-09-14 09:52:37,Burbank,2021-01-01 07:53:00,10.0,33.0,Fair,987.80,6.0,0.0,10.0
